# Задание по программированию: Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. 


In [187]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift, estimate_bandwidth
from geopy.distance import geodesic
from itertools import groupby
from operator import itemgetter
%matplotlib inline

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [136]:
data = pd.read_csv('checkins.dat', header=0, sep='|', low_memory=False)

In [137]:
data = data.iloc[1:]

In [138]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
5,984249,2146840.0,5222.0,,,2012-04-21 17:42:58


In [139]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021967 entries, 1 to 1021967
Data columns (total 6 columns):
   id                    1021967 non-null object
 user_id                 1021966 non-null float64
 venue_id                1021966 non-null float64
     latitude            1021966 non-null object
     longitude           1021966 non-null object
     created_at          1021966 non-null object
dtypes: float64(2), object(4)
memory usage: 46.8+ MB


In [140]:
data.columns = ['id','user_id','venue_id','latitude','longitude','created_at']

In [141]:
data= data.apply(pd.to_numeric, errors='ceries')

In [142]:
data = data.fillna('')
data = data[data.latitude != '']
data = data[data.id != '']
data = data[data.longitude != '']
data = data[data.user_id != '']
data = data[data.venue_id != '']

In [143]:
data.head(10)

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824,5222,38.8951,-77.0364,
4,984234,44652,5222,33.8007,-84.4105,
8,984291,105054,5222,45.5235,-122.676,
10,984318,2.14654e+06,5222,40.7645,-111.905,
11,984232,93870,380645,33.4484,-112.074,
12,984483,1.03029e+06,955969,32.2217,-110.926,
13,984685,304253,23558,40.65,-73.95,
14,984470,720850,749715,33.4484,-112.074,
16,984610,1.63967e+06,442605,33.4148,-111.909,
19,984653,1.64719e+06,23558,42.3584,-71.0598,


In [144]:
data.describe()

,id,user_id,venue_id,latitude,longitude,created_at
count,396634.0,396634.0,396634.0,396634.000000,396634.000000,396634
unique,396634.0,167709.0,47321.0,11785.000000,11731.000000,1
top,927039.0,1326476.0,5222.0,41.878114,-87.629798,
freq,1.0,48.0,12739.0,15254.000000,15254.000000,396634


In [145]:
data = data.drop('id', 1)
data = data.drop('user_id', 1)
data = data.drop('venue_id', 1)
data = data.drop('created_at', 1)

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [123]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 2 to 1021965
Data columns (total 2 columns):
latitude     396634 non-null object
longitude    396634 non-null object
dtypes: object(2)
memory usage: 9.1+ MB


In [124]:
data.head()

,latitude,longitude
2,38.8951,-77.0364
4,33.8007,-84.4105
8,45.5235,-122.676
10,40.7645,-111.905
11,33.4484,-112.074


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [125]:
cluster = MeanShift(bandwidth=0.1)

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.



In [126]:
X = data.iloc[:100000]

In [127]:
cluster.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

In [128]:
big_cluster_labels = [i for i, k in groupby(sorted(cluster.labels_)) if len(list(k)) > 15]

In [129]:
big_cluster_centers = [cluster.cluster_centers_[label] for label in big_cluster_labels]
for c in big_cluster_centers[0:20]:
    print("{},{}".format(c[0], c[1]))

40.71771639727507,-73.9918354198967
33.44943805020126,-112.00213969017547
33.44638027037988,-111.90188756212359
41.87824377967115,-87.62984336226329
37.68868157406161,-122.40933037359147
38.8861652155993,-77.04878333074303
33.357344562325096,-111.82265410760392
33.76663623218336,-84.39328918481657
42.363218639848895,-71.07368760857386
47.60624471741767,-122.33204382627093
36.117229142990276,-115.17107342280688
34.06039755458241,-118.24870902659876
44.97794782033687,-93.2673008852605
30.26718361698159,-97.74311928133027
40.76687624004166,-73.83335349045205
39.735830152625304,-104.98658042770822
39.95168037300773,-75.16273592391683
34.03548695312116,-118.43899771946148
32.98089338217789,-117.07811797821928
32.80302053531547,-96.76989743494408


При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. 

Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

<img src="map.png">

20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [185]:
# office_locations = {
#    "Los Angeles":(33.751277, -118.188740),
#    "Miami":(25.867736, -80.324116),
#    "London":(51.503016, -0.075479),
#    "Amsterdam":(52.378894, 4.885084),
#    "Beijing":(39.366487, 117.036146),
#    "Sydney":(-33.868457, 151.205134)
#    }

In [164]:
offices = [
    [33.751277, -118.188740],
    [25.867736, -80.324116],
    [51.503016, -0.075479],
    [52.378894, 4.885084],
    [39.366487, 117.036146],
    [-33.868457, 151.205134]
]

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

In [165]:
def dist(a, b):
    return geodesic(tuple(a), tuple(b)).meters

In [162]:
pair = (dist(big_cluster_centers[0], offices['Los Angeles']), tuple(big_cluster_centers[0]))

In [166]:
distances = []

for office in offices:
    for center in big_cluster_centers:
        pair = (dist(center, office), tuple(center))
        distances.append(pair)

distances = sorted(distances, key=lambda el: el[0], reverse=False)

In [167]:
distances[0:20]

[(823.4439411475207, (52.37296399032261, 4.892317222580647)),
 (868.7548891021023, (-33.86063042857143, 151.20477592857145)),
 (2499.8389390054376, (25.8456722642857, -80.31889059642857)),
 (3475.8459533280306, (51.502991260887086, -0.12553728870967767)),
 (7473.232780811575, (33.8098779552631, -118.14892380690813)),
 (13989.508526127951, (25.7858124199675, -80.2179380368254)),
 (18446.965858320746, (25.70534972105258, -80.28342873815798)),
 (19965.422193502367, (33.8883253427586, -118.04892817172427)),
 (20083.481829807635, (26.010098249285683, -80.19999058571432)),
 (20972.688719543396, (33.87298601157018, -118.36209114655645)),
 (24618.591405677696, (33.97257482142858, -118.16837066666663)),
 (28509.664311609533, (33.81730643390889, -117.891249170958)),
 (30052.96080939686, (26.138843786842077, -80.33434683684207)),
 (30781.40806039245, (33.98393587403844, -118.00740497307689)),
 (31753.93518744878, (33.67430265976576, -117.85878926777275)),
 (32558.263282541993, (26.120862658633104

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [186]:
print(distances[0][1])

(52.37296399032261, 4.892317222580647)


In [181]:
def write_answer(answers):
    with open("answer.txt", "w") as file_obj:
        file_obj.write(" ".join([str(num) for num in answers]))

In [182]:
answers = [item for item in distances[0][1]]
answers

[52.37296399032261, 4.892317222580647]

In [183]:
write_answer(answers)